In [19]:

import cv2
import numpy as np
from matplotlib import pyplot as plt 
from numpy import asarray
from PIL import Image
import os
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import pickle
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout,BatchNormalization,Activation
from datetime import datetime
import csv

In [20]:
import cv2
import os

# List of folders
folders = ["C:/Users/rodea/OneDrive/Desktop/Dataset/J004","C:/Users/rodea/OneDrive/Desktop/Dataset/J011","C:/Users/rodea/OneDrive/Desktop/Dataset/J012","C:/Users/rodea/OneDrive/Desktop/Dataset/J015",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J016","C:/Users/rodea/OneDrive/Desktop/Dataset/J021","C:/Users/rodea/OneDrive/Desktop/Dataset/J023","C:/Users/rodea/OneDrive/Desktop/Dataset/J024",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J025","C:/Users/rodea/OneDrive/Desktop/Dataset/J031","C:/Users/rodea/OneDrive/Desktop/Dataset/J037","C:/Users/rodea/OneDrive/Desktop/Dataset/J056",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J058","C:/Users/rodea/OneDrive/Desktop/Dataset/J065","C:/Users/rodea/OneDrive/Desktop/Dataset/J066","C:/Users/rodea/OneDrive/Desktop/Dataset/J069",
           "C:/Users/rodea/OneDrive/Desktop/Dataset/J074"]
        
          
          

# Initialize an empty list to store processed images
train_images = []

# Iterate through each folder
for folder in folders:
    # List all files in the folder
    files = os.listdir(folder)

    # Iterate through each file
    for file in files:
        # Get the full file path
        file_path = os.path.join(folder, file)

        # Read the image
        img = cv2.imread(file_path)

        # Process the image (e.g., resize, normalize, etc.)
        processed_img = cv2.resize(img, (200,400))  #
        processed_img = processed_img / 255.0  

        # Add processed image to the list
        train_images.append(processed_img)

# Convert the list of images to a numpy array
concatenated_images = np.array(train_images)

# Print the shape of concatenated images
print("Shape of concatenated images:", concatenated_images.shape)



Shape of concatenated images: (850, 400, 200, 3)


In [21]:

num_classes = 17
labels = y = np.repeat([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],50)


y = tf.keras.utils.to_categorical(labels, num_classes)
print(y[48:52])

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [22]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(400, 200, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(17, activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [24]:
model.fit(concatenated_images,y,epochs =10)

Epoch 1/10
27/27 [==============================] - 178s 6s/step - loss: 1.4365 - accuracy: 0.6024
Epoch 2/10
27/27 [==============================] - 214s 8s/step - loss: 0.2705 - accuracy: 0.9576
Epoch 3/10
27/27 [==============================] - 187s 7s/step - loss: 0.1304 - accuracy: 0.9859
Epoch 4/10
27/27 [==============================] - 171s 6s/step - loss: 0.0588 - accuracy: 1.0000
Epoch 5/10
27/27 [==============================] - 173s 6s/step - loss: 0.0477 - accuracy: 0.9988
Epoch 6/10
27/27 [==============================] - 183s 7s/step - loss: 0.0349 - accuracy: 1.0000
Epoch 7/10
27/27 [==============================] - 520s 20s/step - loss: 0.0294 - accuracy: 0.9988
Epoch 8/10
27/27 [==============================] - 167s 6s/step - loss: 0.0304 - accuracy: 1.0000
Epoch 9/10
27/27 [==============================] - 155s 6s/step - loss: 0.0198 - accuracy: 1.0000
Epoch 10/10
27/27 [==============================] - 161s 6s/step - loss: 0.0217 - accuracy: 1.0000


In [25]:
##storing the model
model.save('complex_cnn')

In [2]:
##LIVE USING CAMERA

import cv2
import pickle
import numpy as np
import os
from datetime import datetime
from keras.models import load_model
import csv


model = load_model('complex_cnn.h5')

def preprocess_image(frame):
    
    processed_image = frame / 255.0  
    return processed_image


attendance = True
cap = cv2.VideoCapture(0)

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

with open('attendance_log.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Time', 'Prediction'])

logged_predictions = set()  

def log_attendance(predicted_class):
    if predicted_class not in logged_predictions:  
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        current_date = now.strftime("%d/%m/%Y")

        with open('attendance_log.csv', mode='a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow([current_date, current_time, predicted_class])

        logged_predictions.add(predicted_class) 

while attendance:
    ret, frame = cap.read()
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        
        padding = 50
        face_center_x = x + w // 2
        face_center_y = y + h // 2
        roi = frame[max(0, face_center_y - h // 2 - padding):min(frame.shape[0], face_center_y + h // 2 + padding),
                    max(0, face_center_x - w // 2 - padding):min(frame.shape[1], face_center_x + w // 2 + padding)]

        resized_face = cv2.resize(roi, (200, 400))

        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        processed_image = resized_face / 255.0  
        prediction = model.predict(np.array([processed_image]))

        class_labels = ['J004-Jay Ajmera','J011-Savali Chavan','J012-Snehee Cheeda','J015 -Farid Damania','J016-Aditya Das','J021- Heet Dhandukia','J023-Anish Gharat','J024-Suchetan Ghosh',
                        'J025-Monish Gosar','J031- Naiktik Jain','J037-Rudra Joshi','J056-Atharva Rode','J058-Jash Shah','J065- Kallind Soni','J066- Naman Upadhyay','J069- Ismail Wangde','J074- Mihir Shah']
        predicted_class = class_labels[np.argmax(prediction)]

        log_attendance(predicted_class)

        annotation = predicted_class
        cv2.putText(frame, annotation, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    frame_height, frame_width, _ = frame.shape
    frame = cv2.resize(frame, (int(frame_width * (800 / frame_height)), 400))
  

    cv2.imshow('Face Detection and Prediction', frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break





1/1 [==============================] - 0s 102ms/step
